In [4]:
import numpy as np
import rootutils

root = rootutils.setup_root(search_from=".", indicator=".git")

DATA_DIR_TRAIN = root / "data" / "DL" / "TRAIN"
DATA_DIR_TEST = root / "data" / "DL" / "TEST"

In [5]:
dl_data_path_train = DATA_DIR_TRAIN / "dl_data_train.npz"
dl_data_path_test = DATA_DIR_TEST / "dl_data_test.npz"
dl_data_train = np.load(dl_data_path_train, allow_pickle=True)
dl_data_test = np.load(dl_data_path_test, allow_pickle=True)

In [7]:
print(dl_data_train)
print("Keys:", dl_data_train.files)
for key in dl_data_train.files:
    print(f"{key}: shape={dl_data_train[key].shape}, dtype={dl_data_train[key].dtype}")

print(dl_data_test)
print("Keys:", dl_data_test.files)
for key in dl_data_test.files:
    print(f"{key}: shape={dl_data_test[key].shape}, dtype={dl_data_test[key].dtype}")

NpzFile '/Users/denizhatemo/Desktop/CDL1Challenge/CDL1-MChallenge/data/DL/TRAIN/dl_data_train.npz' with keys: X, y
Keys: ['X', 'y']
X: shape=(2879, 250, 20), dtype=float32
y: shape=(2879,), dtype=object
NpzFile '/Users/denizhatemo/Desktop/CDL1Challenge/CDL1-MChallenge/data/DL/TEST/dl_data_test.npz' with keys: X, y
Keys: ['X', 'y']
X: shape=(1250, 250, 20), dtype=float32
y: shape=(1250,), dtype=object


CNN
LSTM
MLP

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

# Prepare data
X = dl_data_train['X']  # shape: (N, T, C) or (N, C, T)
y = dl_data_train['y']  # shape: (N,)

# If y is object dtype or string, encode to integer labels
if y.dtype.kind in {'U', 'S', 'O'} or not np.issubdtype(y.dtype, np.integer):
    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    y = le.fit_transform(y)
    print("Label mapping:", dict(zip(le.classes_, le.transform(le.classes_))))
else:
    le = None  # No label encoding needed

if X.shape[1] < X.shape[2]:  # (N, T, C) -> (N, C, T)
    X = np.transpose(X, (0, 2, 1))

X = X.astype(np.float32)
y = y.astype(np.int64)
num_classes = len(np.unique(y))

# Convert to torch tensors
X_tensor = torch.from_numpy(X)
y_tensor = torch.from_numpy(y)

# Split into train, val, test (e.g., 70/15/15)
N = X.shape[0]
n_train = int(0.7 * N)
n_val = int(0.15 * N)
n_test = N - n_train - n_val
dataset = TensorDataset(X_tensor, y_tensor)
train_set, val_set, test_set = random_split(dataset, [n_train, n_val, n_test], generator=torch.Generator().manual_seed(42))

batch_size = 64
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size)
test_loader = DataLoader(test_set, batch_size=batch_size)

# Define a simple 1D CNN
class SimpleCNN(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels, 32, kernel_size=5, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm1d(64)
        self.pool = nn.AdaptiveMaxPool1d(1)
        self.fc = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.pool(x).squeeze(-1)
        x = self.fc(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN(in_channels=X.shape[1], num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Training loop
def train_model(model, train_loader, val_loader, epochs=20):
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            out = model(xb)
            loss = criterion(out, yb)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * xb.size(0)
        train_loss /= len(train_loader.dataset)

        # Validation
        model.eval()
        val_loss = 0
        all_preds, all_labels = [], []
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                out = model(xb)
                loss = criterion(out, yb)
                val_loss += loss.item() * xb.size(0)
                preds = out.argmax(dim=1).cpu().numpy()
                all_preds.append(preds)
                all_labels.append(yb.cpu().numpy())
        val_loss /= len(val_loader.dataset)
        val_acc = accuracy_score(np.concatenate(all_labels), np.concatenate(all_preds))
        print(f"Epoch {epoch+1}/{epochs} - Train loss: {train_loss:.4f} - Val loss: {val_loss:.4f} - Val acc: {val_acc:.4f}")

train_model(model, train_loader, val_loader, epochs=20)

# Evaluation on test set
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for xb, yb in test_loader:
        xb, yb = xb.to(device), yb.to(device)
        out = model(xb)
        preds = out.argmax(dim=1).cpu().numpy()
        all_preds.append(preds)
        all_labels.append(yb.cpu().numpy())
y_true = np.concatenate(all_labels)
y_pred = np.concatenate(all_preds)

acc = accuracy_score(y_true, y_pred)
prec, rec, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
print(f"Test Accuracy: {acc:.4f}")
print(f"Test Precision: {prec:.4f}")
print(f"Test Recall: {rec:.4f}")
print(f"Test F1-score: {f1:.4f}")

# If you want to map predictions back to string labels:
if le is not None:
    y_true_labels = le.inverse_transform(y_true)
    y_pred_labels = le.inverse_transform(y_pred)
    print("Example true labels:", y_true_labels[:10])
    print("Example predicted labels:", y_pred_labels[:10])


Label mapping: {'climbing': np.int64(0), 'joggen': np.int64(1), 'sitting': np.int64(2), 'walking': np.int64(3)}
Epoch 1/20 - Train loss: 0.8301 - Val loss: 0.6546 - Val acc: 0.8445
Epoch 2/20 - Train loss: 0.4598 - Val loss: 0.4258 - Val acc: 0.8585
Epoch 3/20 - Train loss: 0.3745 - Val loss: 0.3824 - Val acc: 0.8677
Epoch 4/20 - Train loss: 0.3490 - Val loss: 0.3568 - Val acc: 0.8608
Epoch 5/20 - Train loss: 0.3300 - Val loss: 0.3451 - Val acc: 0.8770
Epoch 6/20 - Train loss: 0.3233 - Val loss: 0.3490 - Val acc: 0.8677
Epoch 7/20 - Train loss: 0.3087 - Val loss: 0.3268 - Val acc: 0.8654
Epoch 8/20 - Train loss: 0.2796 - Val loss: 0.3037 - Val acc: 0.8886
Epoch 9/20 - Train loss: 0.2687 - Val loss: 0.3056 - Val acc: 0.8770
Epoch 10/20 - Train loss: 0.2575 - Val loss: 0.2848 - Val acc: 0.8840
Epoch 11/20 - Train loss: 0.2440 - Val loss: 0.2782 - Val acc: 0.9002
Epoch 12/20 - Train loss: 0.2408 - Val loss: 0.2614 - Val acc: 0.9026
Epoch 13/20 - Train loss: 0.2300 - Val loss: 0.2706 - Val